# CSC 480-F25 Lab 7: Reasoning and Querying Knowledge Graphs

### Overview

In this lab you will load the case data from Lab 6 into Neo4j and practice reasoning via Cypher queries. You’ll start by verifying a Neo4j connection, read all CSVs into pandas DataFrames, insert nodes and relationships into the graph (manual path), and then optionally use an agentic system to suggest and run useful investigative queries. The goal is to translate investigative questions into concrete queries and interpret the results.

By the end, you should be able to:
- Import entities and relationships from CSVs into a Neo4j graph.
- Formulate and execute Cypher queries to answer investigative questions.
- (Optional) Use an agentic system to propose/select useful queries and run them via a tool.


---

## Part 1: Environment setup

Install Python dependencies used below (pandas, neo4j, and optional agentic libraries). Re-run if the environment changes or packages are missing.

In [1]:
# Environment setup
%pip install pandas neo4j "autogen-core" "autogen-agentchat" "autogen-ext[openai,azure]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


---

### Verify Neo4j connectivity

Run a quick Bolt connection test to ensure your local Neo4j instance is reachable with the credentials above.

In [1]:
# Neo4j connection sanity check
from neo4j import GraphDatabase

username = "neo4j"  # default user
password = "simple123"  # example password used in L6
hostname = "127.0.0.1"
port = 7687
uri = f"bolt://{hostname}:{port}"

try:
    driver = GraphDatabase.driver(uri, auth=(username, password))
    with driver.session() as session:
        result = session.run("RETURN 1 as test")
        print(f"Connection successful! Result: {result.single()['test']}")
    driver.close()
except Exception as e:
    print(f"Connection failed: {type(e).__name__}: {e}")

Connection successful! Result: 1


---

### Load case CSVs into DataFrames

Read all entity and relationship CSVs from the L6-7_data folder and preview a few rows for each. Adjust the path if your data lives elsewhere.

In [2]:
# Read all CSVs into DataFrames (same as L6)
from pathlib import Path
import pandas as pd

# Change this to your data path if needed
data_path = Path("../L6 - Knowledge Graphs/L6-7_data")

csvs = {
    "Person": data_path / "Person.csv",
    "Location": data_path / "Location.csv",
    "Event": data_path / "Event.csv",
    "Evidence": data_path / "Evidence.csv",
    "Case": data_path / "Case.csv",
    "Person_Person_Rel": data_path / "Person_Person_Rel.csv",
    "Person_Location_Rel": data_path / "Person_Location_Rel.csv",
    "Event_Evidence_Location_Rel": data_path / "Event_Evidence_Location_Rel.csv",
    "Case_Related_Rel": data_path / "Case_Related_Rel.csv",
}

frames = {}
for key, path in csvs.items():
    frames[key] = pd.read_csv(path)
    print(f"Loaded {key}: {frames[key].shape[0]} rows")

# Quick peek
for key, df in frames.items():
    print("\n===", key, "===")
    print(df.head(3))

Loaded Person: 9 rows
Loaded Location: 7 rows
Loaded Event: 7 rows
Loaded Evidence: 6 rows
Loaded Case: 1 rows
Loaded Person_Person_Rel: 9 rows
Loaded Person_Location_Rel: 8 rows
Loaded Event_Evidence_Location_Rel: 4 rows
Loaded Case_Related_Rel: 5 rows

=== Person ===
   id           name               type                   status         dob
0  KS  Kristin Smart             Victim  Deceased (Missing Body)  1977-02-20
1  PF    Paul Flores   Suspect/Murderer   Convicted (25 to Life)  1977-04-11
2  RF   Ruben Flores  Suspect/Accessory                Acquitted  1941-01-01

=== Location ===
            id                          name                  type  \
0    PARTY_LOC      Crandall Way Party House  Off-Campus Residence   
1    MUIR_HALL         Muir Hall (KS's Dorm)             Dormitory   
2  SANTA_LUCIA  Santa Lucia Hall (PF's Dorm)             Dormitory   

           address             city  
0              NaN  San Luis Obispo  
1  Cal Poly Campus  San Luis Obispo  
2  Cal Po

---

## Part 2: Cypher Queries

Convenience wrappers to execute single or multi-statement Cypher and to run read-only SELECT queries using the Neo4j Python driver, similar to those used in L6.

In [11]:
# Helper functions for Cypher (schema + select)
from neo4j import GraphDatabase
from typing import List, Dict, Any, Optional

# Reuse the same connection vars defined earlier
_driver = GraphDatabase.driver(uri, auth=(username, password))


def execute_cypher_query(
    query_str: str,
    description: str = "Executing query",
    verbose: bool = False,
    params: Optional[Dict[str, Any]] = None,
) -> str:
    if verbose:
        print(f"\n{'='*80}")
        print(f"EXECUTING CYPHER QUERY: {description}")
        print(f"{'='*80}")
        print(f"Query:\n{query_str}")
        print(f"{'='*80}\n")
    nodes_created = relationships_created = properties_set = labels_added = (
        indexes_added
    ) = constraints_added = 0
    with _driver.session() as session:
        result = session.run(query_str, params or {})
        summary = result.consume()
        counters = summary.counters
        nodes_created += counters.nodes_created
        relationships_created += counters.relationships_created
        properties_set += counters.properties_set
        labels_added += counters.labels_added
        indexes_added += counters.indexes_added
        constraints_added += counters.constraints_added
    response = "\n".join(
        [
            f"Nodes created: {nodes_created}",
            f"Relationships created: {relationships_created}",
            f"Properties set: {properties_set}",
            f"Labels added: {labels_added}",
            f"Indexes added: {indexes_added}",
            f"Constraints added: {constraints_added}",
        ]
    )
    if verbose:
        print(response)
    return response


def execute_multi_cypher(
    script: str,
    description: str = "Executing multi-statement script",
    verbose: bool = True,
):
    # Split on semicolons; ignore comments and empties
    lines = [ln for ln in script.splitlines() if not ln.strip().startswith("//")]
    joined = " ".join(lines)
    stmts = [s.strip() for s in joined.split(";") if s.strip()]
    for i, stmt in enumerate(stmts, 1):
        execute_cypher_query(
            stmt, description=f"{description} (part {i}/{len(stmts)})", verbose=verbose
        )


def run_cypher_select(
    query: str, params: Optional[Dict[str, Any]] = None
) -> List[Dict[str, Any]]:
    with _driver.session() as session:
        result = session.run(query, params or {})
        rows = [dict(r) for r in result]
    return rows


---

### Create/verify constraints and indexes

Create unique id constraints and helpful indexes for faster lookups.

In [12]:
# Ensure constraints and indexes exist
schema_cypher = """
// Node uniqueness
CREATE CONSTRAINT case_id_unique IF NOT EXISTS FOR (c:Case) REQUIRE c.id IS UNIQUE;
CREATE CONSTRAINT event_id_unique IF NOT EXISTS FOR (e:Event) REQUIRE e.id IS UNIQUE;
CREATE CONSTRAINT evidence_id_unique IF NOT EXISTS FOR (ev:Evidence) REQUIRE ev.id IS UNIQUE;
CREATE CONSTRAINT location_id_unique IF NOT EXISTS FOR (l:Location) REQUIRE l.id IS UNIQUE;
CREATE CONSTRAINT person_id_unique IF NOT EXISTS FOR (p:Person) REQUIRE p.id IS UNIQUE;

// Helpful indexes
CREATE INDEX person_name IF NOT EXISTS FOR (p:Person) ON (p.name);
CREATE INDEX location_name IF NOT EXISTS FOR (l:Location) ON (l.name);
CREATE INDEX case_name IF NOT EXISTS FOR (c:Case) ON (c.name);
"""
execute_multi_cypher(schema_cypher, description="Create/verify constraints & indexes", verbose=True)
print("Schema check complete.")


EXECUTING CYPHER QUERY: Create/verify constraints & indexes (part 1/8)
Query:
CREATE CONSTRAINT case_id_unique IF NOT EXISTS FOR (c:Case) REQUIRE c.id IS UNIQUE

Nodes created: 0
Relationships created: 0
Properties set: 0
Labels added: 0
Indexes added: 0
Constraints added: 0

EXECUTING CYPHER QUERY: Create/verify constraints & indexes (part 2/8)
Query:
CREATE CONSTRAINT event_id_unique IF NOT EXISTS FOR (e:Event) REQUIRE e.id IS UNIQUE

Nodes created: 0
Relationships created: 0
Properties set: 0
Labels added: 0
Indexes added: 0
Constraints added: 0

EXECUTING CYPHER QUERY: Create/verify constraints & indexes (part 3/8)
Query:
CREATE CONSTRAINT evidence_id_unique IF NOT EXISTS FOR (ev:Evidence) REQUIRE ev.id IS UNIQUE

Nodes created: 0
Relationships created: 0
Properties set: 0
Labels added: 0
Indexes added: 0
Constraints added: 0

EXECUTING CYPHER QUERY: Create/verify constraints & indexes (part 4/8)
Query:
CREATE CONSTRAINT location_id_unique IF NOT EXISTS FOR (l:Location) REQUIRE l.

---

### Insert the nodes (entities)

Create Person, Location, Event, Evidence, and Case nodes from the corresponding CSV columns using MERGE to keep operations idempotent.

In [ ]:
from datetime import datetime


def dict_rows(df, cols):
    return [  # convert DataFrame rows to dicts with None for NaN
        {c: None if pd.isna(v) else v for c, v in row.items()}
        for row in df[cols].to_dict(orient="records")
    ]


# Person
rows = dict_rows(frames["Person"], ["id", "name", "type", "status", "dob"])
person_q = r"""
UNWIND $rows AS row
MERGE (p:Person {id: row.id})
SET p.name = row.name,
    p.type = row.type,
    p.status = row.status,
    p.dob = row.dob
"""
with _driver.session() as session:
    session.run(person_q, {"rows": rows})
print("Inserted Persons")

# Location
rows = dict_rows(frames["Location"], ["id", "name", "type", "address", "city"])
loc_q = r"""
UNWIND $rows AS row
MERGE (l:Location {id: row.id})
SET l.name = row.name,
    l.type = row.type,
    l.address = row.address,
    l.city = row.city
"""
with _driver.session() as session:
    session.run(loc_q, {"rows": rows})
print("Inserted Locations")

# Event
rows = dict_rows(frames["Event"], ["id", "type", "date", "description"])
event_q = r"""
UNWIND $rows AS row
MERGE (e:Event {id: row.id})
SET e.type = row.type,
    e.date = row.date,
    e.description = row.description
"""
with _driver.session() as session:
    session.run(event_q, {"rows": rows})
print("Inserted Events")

# Evidence
rows = dict_rows(frames["Evidence"], ["id", "type", "status", "description"])
ev_q = r"""
UNWIND $rows AS row
MERGE (ev:Evidence {id: row.id})
SET ev.type = row.type,
    ev.status = row.status,
    ev.description = row.description
"""
with _driver.session() as session:
    session.run(ev_q, {"rows": rows})
print("Inserted Evidence")

# Case
rows = dict_rows(frames["Case"], ["id", "name", "status", "dateOpened"])
case_q = r"""
UNWIND $rows AS row
MERGE (c:Case {id: row.id})
SET c.name = row.name,
    c.status = row.status,
    c.dateOpened = row.dateOpened
"""
with _driver.session() as session:
    session.run(case_q, {"rows": rows})
print("Inserted Cases")

Inserted Persons
Inserted Locations
Inserted Events
Inserted Evidence
Inserted Cases


---

### Insert relationships (edges)

Create typed relationships for Person→Person, Person→Location, Evidence→Location, and Person→Case with relevant properties.

In [8]:
# Insert relationships
# Person-Person
rows = frames["Person_Person_Rel"].to_dict(orient="records")
pp_types = set([r[":TYPE"] for r in rows])
with _driver.session() as session:
    for t in pp_types:
        subset = [r for r in rows if r[":TYPE"] == t]
        q = f"""
        UNWIND $rows AS row
        MATCH (a:Person {{id: row.`:START_ID(Person)`}}), (b:Person {{id: row.`:END_ID(Person)`}})
        MERGE (a)-[r:{t}]->(b)
        SET r.relationshipType = row.relationshipType
        """
        session.run(q, {"rows": subset})
print("Inserted Person-Person relationships")

# Person-Location
rows = frames["Person_Location_Rel"].to_dict(orient="records")
pl_types = set([r[":TYPE"] for r in rows])
with _driver.session() as session:
    for t in pl_types:
        subset = [r for r in rows if r[":TYPE"] == t]
        q = f"""
        UNWIND $rows AS row
        MATCH (p:Person {{id: row.`:START_ID(Person)`}}), (l:Location {{id: row.`:END_ID(Location)`}})
        MERGE (p)-[r:{t}]->(l)
        SET r.date = row.date, r.time = row.time
        """
        session.run(q, {"rows": subset})
print("Inserted Person-Location relationships")

# Evidence-Location (from Event_Evidence_Location_Rel.csv) — evidence to location
rows = frames["Event_Evidence_Location_Rel"].to_dict(orient="records")
el_types = set([r[":TYPE"] for r in rows])
with _driver.session() as session:
    for t in el_types:
        subset = [r for r in rows if r[":TYPE"] == t]
        q = f"""
        UNWIND $rows AS row
        MATCH (ev:Evidence {{id: row.`Event_Evidence_Location_Rel:START_ID`}}), (l:Location {{id: row.`:END_ID`}})
        MERGE (ev)-[r:{t}]->(l)
        SET r.date = row.date
        """
        session.run(q, {"rows": subset})
print("Inserted Evidence-Location relationships")

# Person-Case
rows = frames["Case_Related_Rel"].to_dict(orient="records")
pc_types = set([r[":TYPE"] for r in rows])
with _driver.session() as session:
    for t in pc_types:
        subset = [r for r in rows if r[":TYPE"] == t]
        q = f"""
        UNWIND $rows AS row
        MATCH (p:Person {{id: row.`:START_ID(Person)`}}), (c:Case {{id: row.`:END_ID(Case)`}})
        MERGE (p)-[r:{t}]->(c)
        SET r.outcome = row.outcome
        """
        session.run(q, {"rows": subset})
print("Inserted Person-Case relationships")

Inserted Person-Person relationships
Inserted Person-Location relationships
Inserted Evidence-Location relationships
Inserted Person-Case relationships


---

### Sanity checks

Verify node and relationship counts to ensure the import completed as expected. These should be non-zero values.

In [9]:
# Quick counts after import
counts = {
    "Person": run_cypher_select("MATCH (n:Person) RETURN count(n) AS c")[0]["c"],
    "Location": run_cypher_select("MATCH (n:Location) RETURN count(n) AS c")[0]["c"],
    "Event": run_cypher_select("MATCH (n:Event) RETURN count(n) AS c")[0]["c"],
    "Evidence": run_cypher_select("MATCH (n:Evidence) RETURN count(n) AS c")[0]["c"],
    "Case": run_cypher_select("MATCH (n:Case) RETURN count(n) AS c")[0]["c"],
}
print(counts)

rel_counts = run_cypher_select("""
MATCH ()-[r]->() RETURN type(r) AS rel, count(*) AS c ORDER BY c DESC
""")
for row in rel_counts:
    print(row)

{'Person': 9, 'Location': 7, 'Event': 7, 'Evidence': 6, 'Case': 1}
{'rel': 'ACCOMPANIED_BY', 'c': 5}
{'rel': 'ATTENDED_PARTY_AT', 'c': 4}
{'rel': 'FAMILY_RELATIONSHIP', 'c': 4}
{'rel': 'SEIZED_FROM', 'c': 3}
{'rel': 'RESIDENCE_OF', 'c': 2}
{'rel': 'FILED_CIVIL_SUIT_IN', 'c': 2}
{'rel': 'LAST_SEEN_NEAR', 'c': 1}
{'rel': 'VICTIM_IN', 'c': 1}
{'rel': 'LIVED_AT', 'c': 1}
{'rel': 'SUSPECT_IN', 'c': 1}
{'rel': 'ACCUSED_IN', 'c': 1}
{'rel': 'FOUND_AT', 'c': 1}


---

## Part 3: Agentic querying (optional)

Use the AutoGen-based agentic system to propose and execute useful investigative Cypher queries via a tool. Again, this is mostly for fun, and not guaranteed to converge.

In [14]:
# Optional: Agentic query system (AutoGen + Azure OpenAI)
import os
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.base import TaskResult
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT", "gpt-5-mini")
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-12-01-preview")
azure_endpoint = os.getenv(
    "AZURE_OPENAI_ENDPOINT", "https://your-resource.openai.azure.com/"
)
api_key = os.getenv("AZURE_SUBSCRIPTION_KEY")

if not api_key:
    raise Exception(
        "AZURE_SUBSCRIPTION_KEY is not set. Agentic section will be a no-op."
    )

client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_deployment,
    model="gpt-5-mini",
    api_version=api_version,
    azure_endpoint=azure_endpoint,
    api_key=api_key,
)


# Tool to run SELECT queries and return rows
def cypher_select_tool(query_str: str, description: str = "Run read query"):
    print(f"[TOOL] {description}\n{query_str}")
    rows = run_cypher_select(query_str)
    # Truncate for readability
    preview = rows[:10]
    print(f"[RESULT] {len(rows)} rows; preview: {preview}")
    return {"row_count": len(rows), "rows": preview}


planner_prompt = """
You are a Query Planner for a Neo4j knowledge graph of the Kristin Smart case. 
Propose and run 3-5 SELECT queries that help an investigator, such as:
- Timeline of key events with dates
- Last known locations of the victim
- Evidence found at or seized from specific locations (e.g., RF_HOME, PF_HOME_LA)
- Relationships between suspects and family members
- Who accompanied whom on the night of disappearance
Use the query tool to execute your queries, summarize results briefly, and end with DONE.
"""

if client:
    planner = AssistantAgent(
        name="QueryPlanner",
        system_message=planner_prompt,
        model_client=client,
        tools=[cypher_select_tool],
    )

    termination = TextMentionTermination("DONE")
    group = RoundRobinGroupChat(
        [planner], max_turns=10, termination_condition=termination
    )

    task = (
        "Plan useful investigative queries over the graph and execute each using the tool. "
        "Provide a one-line summary after each tool call."
    )

    result: TaskResult = await group.run(task=task)
    for m in result.messages:
        print(m.content)
else:
    print(
        "Agentic query system not initialized (missing Azure credentials). You may skip or set AZURE_SUBSCRIPTION_KEY."
    )

[TOOL] Timeline of key events with dates
MATCH (e:Event)
WHERE exists(e.date)
RETURN e.date AS date, e.type AS event_type, e.name AS event_name, e.description AS description, e.location AS location
ORDER BY e.date ASC
LIMIT 200


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: location)} {position: line: 3, column: 100, offset: 140} for query: 'MATCH (e:Event)\nWHERE e.date IS NOT NULL\nRETURN e.date AS date, e.type AS event_type, e.name AS event_name, e.description AS description, e.location AS location\nORDER BY e.date ASC\nLIMIT 200'


[TOOL] Timeline of key events with dates
MATCH (e:Event)
WHERE e.date IS NOT NULL
RETURN e.date AS date, e.type AS event_type, e.name AS event_name, e.description AS description, e.location AS location
ORDER BY e.date ASC
LIMIT 200
[RESULT] 7 rows; preview: [{'date': '1996-05-25 02:00:00', 'event_type': 'Last Sighting/Disappearance', 'event_name': None, 'description': 'Kristin was last seen with Paul Flores near his dorm.', 'location': None}, {'date': '2002-05-25', 'event_type': 'Legally Declared Dead', 'event_name': None, 'description': 'Declared dead on the 6th anniversary of her disappearance.', 'location': None}, {'date': '2016-09-06', 'event_type': 'Cal Poly Excavation', 'event_name': None, 'description': "Sheriff's Office and FBI dug on Cal Poly hillside.", 'location': None}, {'date': '2021-03-15', 'event_type': 'Search Warrant (Ruben Flores)', 'event_name': None, 'description': 'Cadaver dogs and GPR used under the deck.', 'location': None}, {'date': '2021-04-13', 'event_type': '

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: LAST_KNOWN_LOCATION)} {position: line: 1, column: 21, offset: 20} for query: 'MATCH (p:Person)-[r:LAST_KNOWN_LOCATION]->(loc:Location)\nRETURN p.name AS person, loc.name AS location, r.date AS date, r.details AS details\nORDER BY r.date DESC\nLIMIT 50'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the

[TOOL] Last known locations of key persons (victim / suspects)
MATCH (p:Person)-[r:LAST_KNOWN_LOCATION]->(loc:Location)
RETURN p.name AS person, loc.name AS location, r.date AS date, r.details AS details
ORDER BY r.date DESC
LIMIT 50
[RESULT] 0 rows; preview: []
[TOOL] Last seen locations and accompaniment relationships
MATCH (p:Person)-[r:LAST_SEEN_AT|:LAST_KNOWN_LOCATION|:SEEN_WITH|:ACCOMPANIED_BY]->(locOrPerson)
RETURN p.name AS subject, type(r) AS relationship, CASE WHEN (locOrPerson:Location) THEN locOrPerson.name ELSE locOrPerson.name END AS object, r.date AS date, r.details AS details
ORDER BY r.date DESC
LIMIT 100
[TOOL] Last seen locations and accompaniment relationships
MATCH (p:Person)-[r:LAST_SEEN_AT|:LAST_KNOWN_LOCATION|:SEEN_WITH|:ACCOMPANIED_BY]->(locOrPerson)
RETURN p.name AS subject, type(r) AS relationship, CASE WHEN (locOrPerson:Location) THEN locOrPerson.name ELSE locOrPerson.name END AS object, r.date AS date, r.details AS details
ORDER BY r.date DESC
LIMIT 100


...

[TOOL] Evidence seized from specific locations like RF_HOME, PF_HOME_LA
MATCH (loc:Location)-[s:SEIZED_EVIDENCE|HAS_EVIDENCE|EVIDENCE_FROM]->(e:Evidence)
RETURN loc.name AS location, type(s) AS relationship, e.id AS evidence_id, e.type AS evidence_type, e.description AS description, s.date AS date
ORDER BY s.date DESC
LIMIT 50
[RESULT] 0 rows; preview: []


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: RELATED_TO)} {position: line: 1, column: 21, offset: 20} for query: 'MATCH (s:Person)-[r:RELATED_TO|FAMILY|ASSOCIATED_WITH]->(o:Person)\nRETURN s.name AS subject, type(r) AS relation, o.name AS object, r.details AS details\nLIMIT 200'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the da

[TOOL] Relationships between suspects and family members
MATCH (s:Person)-[r:RELATED_TO|FAMILY|ASSOCIATED_WITH]->(o:Person)
RETURN s.name AS subject, type(r) AS relation, o.name AS object, r.details AS details
LIMIT 200
[RESULT] 0 rows; preview: []


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: WITH)} {position: line: 1, column: 22, offset: 21} for query: 'MATCH (p1:Person)-[r:WITH]->(p2:Person)\nRETURN p1.name AS person1, type(r) AS relationship, p2.name AS person2, r.date AS date, r.details AS details\nLIMIT 200'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you did

[TOOL] Who accompanied whom on the night of disappearance
MATCH (p1:Person)-[r:WITH]->(p2:Person)
RETURN p1.name AS person1, type(r) AS relationship, p2.name AS person2, r.date AS date, r.details AS details
LIMIT 200
[RESULT] 0 rows; preview: []
Plan useful investigative queries over the graph and execute each using the tool. Provide a one-line summary after each tool call.
[FunctionCall(id='call_rOKwYr2WwJ10TSVd1Gvrzp80', arguments='{"query_str":"MATCH (e:Event)\\nWHERE exists(e.date)\\nRETURN e.date AS date, e.type AS event_type, e.name AS event_name, e.description AS description, e.location AS location\\nORDER BY e.date ASC\\nLIMIT 200","description":"Timeline of key events with dates"}', name='cypher_select_tool')]
[FunctionExecutionResult(content='{code: Neo.ClientError.Statement.SyntaxError} {message: The property existence syntax `... exists(variable.property)` is no longer supported. Please use `variable.property IS NOT NULL` instead. (line 2, column 7 (offset: 22))\n"WHERE exi

---

## Part 4: Reflection

##### Manual Cypher vs. Agentic Querying

_(Compare control/transparency, speed/latency, reproducibility, and correctness. When would you prefer writing queries by hand, and when might agentic planning help?)_

##### Were there any query patterns you discovered

_(E.g., OPTIONAL MATCH for sparse data, aggregations for timelines, path patterns for relationship exploration, parameterized queries for reuse.)_
